In [ ]:
# -*- coding: utf-8 -*-
import sys
import numpy as np
import pandas as pd
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore

shift = 100
n = 50       # plotCount
sample=100       # skipData
skip=1000
flg_pause = False
flg_close = False
flg_skip = False

# ボタン関連
def Pause_sw():
    global flg_pause
    if flg_pause == True:flg_pause=False
    elif flg_pause==False:flg_pause=True

def Close_sw():
    global flg_close
    if flg_close == True:flg_close=False
    elif flg_close==False:flg_close=True

def Skip_sw():
    global flg_skip
    if flg_skip==True:flg_skip=False
    elif flg_skip==False:flg_skip=True

def Upd_Plot(item,xlist,ylist,color):
    item.plot(xlist,ylist, clear = True, pen=color)

## ファイル選択ダイアログ
def FileSelectDialog(path='',multi=False):
    import tkinter
    from tkinter import filedialog
    from tkinter import messagebox

    root = tkinter.Tk()
    root.withdraw()

    ftype=[('Csv file','*.csv'),('All files','*.*')]
    ftitle="処理するデータファイルを選択して下さい"
    defext='.csv'

    if len(path) != 0:
        idir = path.replace("\\","/")
    else:
        idir='c:/'
    filename = filedialog.askopenfilename(initialdir=idir,defaultextension=defext,title=ftitle,multiple=multi,filetypes=ftype)

    if filename:
        #tkMessageBox.showinfo('選択したファイル', filename)
        fpath = filename
    else:
        fpath = ""

    return fpath #.replace("u'","").replace("'","")
    
def main():
    global shift,flg_pause,flg_close,n,sample,flg_skip,skip

    argvs = sys.argv  # コマンドライン引数を格納したリストの取得
    argc = len(argvs) # 引数の個数
    print("EXEC||"+argvs[0])
    fname = FileSelectDialog(path="/Users/jins/Downloads")
    print('SOURCE||'+fname)
    df = pd.read_csv(fname, delimiter=',', header=0,parse_dates=[1], encoding="shift-jis")
    d=np.array(df)

    x=np.array(df[u'accX'])
    y=np.array(df[u'accY'])
    z=np.array(df[u'accZ'])
    try:
        cnt=np.array(df[u'DataCount'])
    except:
        cnt=np.array(df['Count'])
    pitch_p=np.array(df[u'Roll_Stab_Point'])
    roll_p=np.array(df[u'Pitch_Stab_Point'])
    balance=np.array(df[u'LR-Balance%'])
    pace=np.array(df[u'Pace-20Steps'])
    bp=np.array(df[u'TotalPoints'])
    BaseLine_x=0
    BaseLine_Pace = df[u'Pace-BaseLine'][0]+0.25

    ################# Param set ###################
    BaseLine_Balance = 0.03
    BreakPoints_Limit = 3.0
##    BaseLine_y=df[u'Pace-BaseLine'][0] #5.8
    Margin = 0.25
    Margin_Pitch = 1.5 # Pitch判定の基準値±マージン度
    Margin_Roll = 1.5  # Roll判定の基準値±マージン度
    Margin_accZ = 0.05 # Balance判定の+マージン%
    Margin_Stability = 2.0  # 姿勢判定の基準値±マージン度
    Margin_Pace = 0.5 # ペース基準値+マージン分 ex)0.25min=15sec
    ###############################################

    ymin,ymax = -128,127
    xmin,xmax = -128,127

    ### アプリケーション作成
    app = QtGui.QApplication([])
    app.quitOnLastWindowClosed()
    ### メインウィンドウ
    mainWindow = QtGui.QMainWindow()
    mainWindow.setWindowTitle("DataPlotter : " + fname) # Title
    mainWindow.resize(1200, 800) # Size
    ### キャンパス
    centralWid = QtGui.QWidget()
    mainWindow.setCentralWidget(centralWid)
    ### レイアウト
    grid = QtGui.QGridLayout()
    centralWid.setLayout(grid)

    ### 表示用ウィジット 1
    Wid1 = pg.PlotWidget(name="accX-Y")
    item1 = Wid1.getPlotItem()
    item1.addLegend()
    item1.setMouseEnabled(x=True,y=True)
    item1.setYRange(ymin, ymax)
    item1.setXRange(xmin, xmax, padding = 0)
    item1.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item1.getAxis("bottom")
    specAxis.setLabel("accX-Y")

    ### 表示用ウィジット 2
    Wid2 = pg.PlotWidget(name="accX-Z")
    item2 = Wid2.getPlotItem()
    item2.addLegend()
    item2.setMouseEnabled(x=True,y=True)
    item2.setYRange(ymin, ymax)
    item2.setXRange(xmin, xmax, padding = 0)
    item2.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item2.getAxis("bottom")
    specAxis.setLabel("accX-Z")

    ### 表示用ウィジット 3
    Wid3 = pg.PlotWidget(name="accZ-Y")
    item3 = Wid3.getPlotItem()
    item3.addLegend()
    item3.setMouseEnabled(x=True,y=True)
    item3.setYRange(ymin, ymax)
    item3.setXRange(xmin, xmax, padding = 0)
    item3.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item3.getAxis("bottom")
    specAxis.setLabel("accZ-Y")

    ### 表示用ウィジット 4
    Wid4 = pg.PlotWidget(name="Pace")
    item4 = Wid4.getPlotItem()
    item4.addLegend()
    item4.setMouseEnabled(x=True,y=True)
    item4.setYRange(0, 15)
    item4.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item4.getAxis("bottom")
    specAxis.setLabel("Pace")

    ### 表示用ウィジット 5
    Wid5 = pg.PlotWidget(name="TotalPoints")
    item5 = Wid5.getPlotItem()
    item5.addLegend()
    item5.setMouseEnabled(x=True,y=True)
    item5.setYRange(0, 4)
    item5.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item5.getAxis("bottom")
    specAxis.setLabel("TotalPoints")

    ### 表示用ウィジット 6
    Wid6 = pg.PlotWidget(name="LR-balance")
    item6 = Wid6.getPlotItem()
    item6.addLegend()
    item6.setMouseEnabled(x=True,y=True)
    item6.setXRange(0.1, -0.1)
    item6.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item6.getAxis("bottom")
    specAxis.setLabel("LR-Balance%")

    ### 表示用ウィジット 7
    Wid7 = pg.PlotWidget(name="Stability-Roll")
    item7 = Wid7.getPlotItem()
    item7.addLegend()
    item7.setMouseEnabled(x=True,y=True)
    item7.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item7.getAxis("bottom")
    specAxis.setLabel("Posture:Pitch-Roll")

    ### 表示用ウィジット 8
    Wid8 = pg.PlotWidget(name="Stability-Pitch")
    item8 = Wid8.getPlotItem()
    item8.addLegend()
    item8.setMouseEnabled(x=True,y=True)
    item8.showGrid(x=True,y=True,alpha=60)
    ### Axis
    specAxis = item8.getAxis("bottom")
    specAxis.setLabel("Posture:Pitch-Pitch")

    ### スタートストップボタン
    btn1 = QtGui.QPushButton('Pause/ReStart')
    btn1.clicked.connect(Pause_sw)
    btn2 = QtGui.QPushButton('Close')
    btn2.clicked.connect(Close_sw)
    btn3 = QtGui.QPushButton('skip-1000cnt')
    btn3.clicked.connect(Skip_sw)

    ### キャンパスにのせる
    grid.addWidget(Wid1,0,0)
    grid.addWidget(Wid2,1,0)
    grid.addWidget(Wid3,0,1)
    grid.addWidget(Wid4,1,1)
    grid.addWidget(Wid5,0,2)
    grid.addWidget(Wid6,1,2)
    grid.addWidget(Wid7,0,3)
    grid.addWidget(Wid8,1,3)
    grid.addWidget(btn1,2,1)
    grid.addWidget(btn2,2,2)
    grid.addWidget(btn3,2,0)

    ### ウィンドウ表示
    pg.setConfigOptions(antialias=True)
    mainWindow.showMaximized()
    item1.plot(x,y,name="accX-Y")
    item2.plot(x,z,name="accX-Z")
    item3.plot(y,z,name="accY-Z")
    item4.plot(cnt,pace,clear=True,pen=[0,210,255],fillLevel=0, fillBrush=(0,210,255,90),name='Pace')
    item4.plot(cnt,np.array(df[u'Pace-BaseLine']), pen=pg.mkPen(color=(255,0,0),width=2),name='BaseLine')
    item5.plot(cnt,bp, clear = True, pen=[255,132,0],fillLevel=0, fillBrush=(255,132,0,100),name='BreakPoints')
    item5.plot(cnt,np.ones(len(df))*BreakPoints_Limit,pen='y',name="LimitLine")
    item6.plot(balance,cnt, clear = True, pen=pg.mkPen(color=(255,0,216),width=2),name='LR=Balance%')
    item6.plot(np.ones(len(df))*(Margin_accZ*-1),cnt,pen='y',name="Lower-LimitLine")
    item6.plot(np.ones(len(df))*Margin_accZ,cnt,pen='y',name="Upper-LimitLine")
    item7.plot(np.array(df[u'Roll_Stab_Point']),np.array(df[u'DataCount']), pen=pg.mkPen(color=(255,100,100),width=2),name='Posture-Roll')
    item7.plot(np.ones(len(df))*(df[u'BaseLine_Roll_Stab'][0]+Margin_Stability),np.array(df[u'DataCount']), pen='y',name='Upper-LimitLine')
    item7.plot(np.ones(len(df))*(df[u'BaseLine_Roll_Stab'][0]-Margin_Stability),np.array(df[u'DataCount']), pen='y',name='Lower-LimitLine')
    item8.plot(np.array(df[u'DataCount']),np.array(df[u'Pitch_Stab_Point']), pen=pg.mkPen(color=(255,100,100),width=2),name='Posture-Pitch')
    item8.plot(np.ones(len(df))*(df[u'BaseLine_Pitch_Stab'][0]+Margin_Stability), pen='y',name='Upper-LimitLine')
    item8.plot(np.ones(len(df))*(df[u'BaseLine_Pitch_Stab'][0]-Margin_Stability), pen='y',name='Lower-LimitLine')

    ### Update
    i=0
    while 1:
        if i>=len(df): #break
            msgbox = QtGui.QMessageBox.question(QtGui.QWidget(),'End Of Data','Replay?',QtGui.QMessageBox.Yes | QtGui.QMessageBox.No,QtGui.QMessageBox.No)
            if msgbox ==QtGui.QMessageBox.Yes:
                i = 0
            else:
                break
        if flg_close==True :break
        if flg_pause == False :
            i+=sample
            if flg_skip == True:
                i+=skip
                flg_skip = False
            item1.plot(x[i-n:i],y[i-n:i], clear = True, pen=pg.mkPen(color=(255, 0, 0), width=2)) #,style=QtCore.Qt.DotLine))
            item2.plot(x[i-n:i],z[i-n:i], clear = True, pen=pg.mkPen(color=(0, 255, 0), width=2))
            item3.plot(y[i-n:i],z[i-n:i], clear = True, pen=pg.mkPen(color=(0, 0, 255), width=2))
            item4.setXRange(i-n*sample, i, padding = 0)
            item5.setXRange(i-n*sample, i, padding = 0)
            item6.setYRange(i-n*sample, i, padding = 0)
            item7.setYRange(i-n*sample, i, padding = 0)
            item8.setXRange(i-n*sample, i, padding = 0)
##            item7.plot(np.array(df[u'DataCount'][:i]),np.array(df[u'Pitch_Stab_Point'][:i]), pen='r')
        QtGui.QApplication.processEvents()

if __name__ == "__main__":
    main()

Usage: # python /Users/jins/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py <filename>
       <filename> is Output from RunApp_algo.py
EXEC||/Users/jins/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py
SOURCE||/Users/jins/Downloads/run_20151008184129_fix_point_result.csv
